In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 30.8 MB/s eta 0:00:00


In [4]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176720 sha256=97957f1efb906f67db19cf1da9a74ee0760b6cd956fcb11ac55eca7fa8fb1eff
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-c

In [6]:
from rdkit import Chem
from rdkit.Chem import PandasTools
from mordred import Calculator,descriptors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [36]:
bbb=pd.read_csv('/content/B3DB_classification.tsv',delimiter='\t')

In [37]:
bbb

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,BBB-,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,BBB-,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,4,bbcpd11 (cimetidine analog) (y-g13),2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,BBB-,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,BBB-,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7802,7803,antazoline,"n-benzyl-n-(4,5-dihydro-1h-imidazol-2-ylmethyl...",c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,2200.0,NaN,BBB-,InChI=1S/C17H19N3/c1-3-7-15(8-4-1)13-20(14-17-...,NaN,R14|R23|R50|,D,NaN
7803,7804,emedastine,"1-(2-ethoxyethyl)-2-(4-methyl-1,4-diazepan-1-y...",CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,3219.0,NaN,BBB+,InChI=1S/C17H26N4O/c1-3-22-14-13-21-16-8-5-4-7...,NaN,R15|R23|R50|,D,NaN
7804,7805,ketotifen,2-(1-methylpiperidin-4-ylidene)-6-thiatricyclo...,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,3827.0,NaN,BBB+,InChI=1S/C19H19NOS/c1-20-9-6-13(7-10-20)18-15-...,NaN,R15|R27|R50|,D,NaN
7805,7806,milrinone,6-methyl-2-oxo-5-pyridin-4-yl-1h-pyridine-3-ca...,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,4197.0,NaN,BBB-,InChI=1S/C12H9N3O/c1-8-11(9-2-4-14-5-3-9)6-10(...,NaN,R15|R23|R27|,D,NaN


In [11]:
PandasTools.AddMoleculeColumnToFrame(bbb,smilesCol='SMILES')
molecules=bbb['ROMol']

In [12]:
calc=Calculator(descriptors,ignore_3D=True)

In [13]:
descriptor_bbb=calc.pandas(molecules)

  1%|          | 58/7807 [00:18<34:25,  3.75it/s]  

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  2%|▏         | 188/7807 [00:54<44:23,  2.86it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 7807/7807 [37:15<00:00,  3.49it/s]


In [32]:
descriptor_bbb

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,1,0,35.289886,2.38053,4.761059,35.289886,1.260353,4.249578,...,10.171643,63.201012,398.068491,9.477821,2428,42,144.0,165.0,9.590278,6.097222
1,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,4,0,45.430282,2.648849,5.297577,45.430282,1.261952,4.519812,...,10.893010,87.033695,520.101247,9.287522,4114,62,194.0,237.0,13.756944,7.916667
2,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,16.678194,2.425683,4.851365,16.678194,1.282938,3.505118,...,9.618402,44.825548,304.910439,16.939469,218,21,68.0,81.0,4.805556,2.861111
3,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,3,23.641772,2.237342,4.474683,23.641772,1.244304,3.806689,...,9.201199,50.610337,341.030979,9.743742,898,22,82.0,88.0,6.583333,4.694444
4,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,1,1,43.435426,2.709582,5.329713,43.435426,1.316225,4.477453,...,10.992336,85.350582,461.168581,7.686143,2850,74,198.0,254.0,11.229167,6.847222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,2,26.938755,2.326623,4.635214,26.938755,1.346938,3.92396,...,9.582249,66.060685,265.157898,6.798920,829,24,100.0,113.0,4.444444,4.527778
7803,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,28.885493,2.4832,4.768669,28.885493,1.312977,4.018574,...,9.863134,70.340391,302.210661,6.296055,1041,35,112.0,131.0,6.166667,5.027778
7804,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,29.305963,2.511472,4.918238,29.305963,1.332089,4.053092,...,10.124268,70.590222,309.118735,7.539481,917,40,122.0,148.0,5.888889,4.750000
7805,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,20.388826,2.390361,4.780722,20.388826,1.274302,3.691943,...,9.618801,48.335122,211.074562,8.442982,425,24,80.0,93.0,5.555556,3.666667


In [38]:
bbb=pd.concat([bbb,descriptor_bbb],axis=1)

In [39]:
bbb

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,...,10.171643,63.201012,398.068491,9.477821,2428,42,144.0,165.0,9.590278,6.097222
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,BBB-,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,...,10.893010,87.033695,520.101247,9.287522,4114,62,194.0,237.0,13.756944,7.916667
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,BBB-,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,...,9.618402,44.825548,304.910439,16.939469,218,21,68.0,81.0,4.805556,2.861111
3,4,bbcpd11 (cimetidine analog) (y-g13),2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,BBB-,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,...,9.201199,50.610337,341.030979,9.743742,898,22,82.0,88.0,6.583333,4.694444
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,BBB-,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,...,10.992336,85.350582,461.168581,7.686143,2850,74,198.0,254.0,11.229167,6.847222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,7803,antazoline,"n-benzyl-n-(4,5-dihydro-1h-imidazol-2-ylmethyl...",c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,2200.0,NaN,BBB-,InChI=1S/C17H19N3/c1-3-7-15(8-4-1)13-20(14-17-...,NaN,R14|R23|R50|,...,9.582249,66.060685,265.157898,6.798920,829,24,100.0,113.0,4.444444,4.527778
7803,7804,emedastine,"1-(2-ethoxyethyl)-2-(4-methyl-1,4-diazepan-1-y...",CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,3219.0,NaN,BBB+,InChI=1S/C17H26N4O/c1-3-22-14-13-21-16-8-5-4-7...,NaN,R15|R23|R50|,...,9.863134,70.340391,302.210661,6.296055,1041,35,112.0,131.0,6.166667,5.027778
7804,7805,ketotifen,2-(1-methylpiperidin-4-ylidene)-6-thiatricyclo...,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,3827.0,NaN,BBB+,InChI=1S/C19H19NOS/c1-20-9-6-13(7-10-20)18-15-...,NaN,R15|R27|R50|,...,10.124268,70.590222,309.118735,7.539481,917,40,122.0,148.0,5.888889,4.750000
7805,7806,milrinone,6-methyl-2-oxo-5-pyridin-4-yl-1h-pyridine-3-ca...,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,4197.0,NaN,BBB-,InChI=1S/C12H9N3O/c1-8-11(9-2-4-14-5-3-9)6-10(...,NaN,R15|R23|R27|,...,9.618801,48.335122,211.074562,8.442982,425,24,80.0,93.0,5.555556,3.666667


In [48]:
bbb['BBB+/BBB-']= bbb['BBB+/BBB-'].replace({'BBB+':1,'BBB-':0})

In [49]:
numeric_bbb = bbb.select_dtypes(include=['number'])

In [50]:
numeric_bbb

,NO.,CID,logBB,BBB+/BBB-,threshold,nAcid,nBase,nAromAtom,nAromBond,nAtom,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,1,5339.0,-2.69,0,NaN,1,0,18,18,42,...,0.000000,10.171643,63.201012,398.068491,9.477821,2428,42,144.0,165.0,6.097222
1,2,3889.0,-2.52,0,NaN,4,0,11,11,56,...,6.529419,10.893010,87.033695,520.101247,9.287522,4114,62,194.0,237.0,7.916667
2,3,2788.0,-2.40,0,NaN,0,0,10,11,18,...,0.000000,9.618402,44.825548,304.910439,16.939469,218,21,68.0,81.0,2.861111
3,4,14022517.0,-2.15,0,NaN,0,3,6,6,35,...,0.000000,9.201199,50.610337,341.030979,9.743742,898,22,82.0,88.0,4.694444
4,5,18595497.0,-2.15,0,NaN,1,1,6,6,60,...,7.309881,10.992336,85.350582,461.168581,7.686143,2850,74,198.0,254.0,6.847222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,7803,2200.0,NaN,0,NaN,0,2,12,12,39,...,6.259581,9.582249,66.060685,265.157898,6.798920,829,24,100.0,113.0,4.527778
7803,7804,3219.0,NaN,1,NaN,0,1,9,10,48,...,7.181592,9.863134,70.340391,302.210661,6.296055,1041,35,112.0,131.0,5.027778
7804,7805,3827.0,NaN,1,NaN,0,1,11,11,41,...,6.880384,10.124268,70.590222,309.118735,7.539481,917,40,122.0,148.0,4.750000
7805,7806,4197.0,NaN,0,NaN,0,0,12,12,25,...,0.000000,9.618801,48.335122,211.074562,8.442982,425,24,80.0,93.0,3.666667


In [51]:
numeric_bbb_cleaned = numeric_bbb.dropna(axis=1)

In [52]:
numeric_bbb_cleaned

,NO.,BBB+/BBB-,nAcid,nBase,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,1,0,1,0,18,18,42,28,0,0,...,0.000000,10.171643,63.201012,398.068491,9.477821,2428,42,144.0,165.0,6.097222
1,2,0,4,0,11,11,56,36,0,0,...,6.529419,10.893010,87.033695,520.101247,9.287522,4114,62,194.0,237.0,7.916667
2,3,0,0,0,10,11,18,13,0,0,...,0.000000,9.618402,44.825548,304.910439,16.939469,218,21,68.0,81.0,2.861111
3,4,0,0,3,6,6,35,19,0,0,...,0.000000,9.201199,50.610337,341.030979,9.743742,898,22,82.0,88.0,4.694444
4,5,0,1,1,6,6,60,33,1,2,...,7.309881,10.992336,85.350582,461.168581,7.686143,2850,74,198.0,254.0,6.847222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,7803,0,0,2,12,12,39,20,0,0,...,6.259581,9.582249,66.060685,265.157898,6.798920,829,24,100.0,113.0,4.527778
7803,7804,1,0,1,9,10,48,22,0,0,...,7.181592,9.863134,70.340391,302.210661,6.296055,1041,35,112.0,131.0,5.027778
7804,7805,1,0,1,11,11,41,22,0,0,...,6.880384,10.124268,70.590222,309.118735,7.539481,917,40,122.0,148.0,4.750000
7805,7806,0,0,0,12,12,25,16,0,0,...,0.000000,9.618801,48.335122,211.074562,8.442982,425,24,80.0,93.0,3.666667


In [58]:
X=numeric_bbb_cleaned.drop(columns=['BBB+/BBB-'])

In [59]:
y=numeric_bbb_cleaned['BBB+/BBB-']

In [60]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [61]:
clf=RandomForestClassifier(n_estimators=100,random_state=42)

In [62]:
clf.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [63]:
y_pred=clf.predict(X_test)

In [64]:
y_prob=clf.predict_proba(X_test)[:,1]

In [65]:
accuracy=accuracy_score(y_test,y_pred)

In [66]:
accuracy

0.9014084507042254

In [67]:
roc_auc=roc_auc_score(y_test,y_prob)

In [68]:
roc_auc

0.9675440424103525

In [69]:
conf_matrix=confusion_matrix(y_test,y_pred)

In [74]:
print(conf_matrix)

[[470  91]
 [ 63 938]]


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [4, 5, 6, 7, 8],
    'criterion': ['gini', 'entropy']
}

# Initialize the Random Forest Classifier
rfc = RandomForestClassifier()

# Set up Grid Search
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor